In [1]:
import tensorflow as tf
from tensorflow import keras

from collections import deque
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import gym

In [2]:
print(tf.__version__)

2.10.0


## DQN (Deep Q Network)

In this lab, we will apply deep learning as function approximations in reinforcement learning. 

Reference: DQN https://arxiv.org/abs/1312.5602

In tabular Q-learning, we maintain a table of state-action pairs $(s,a)$ and save one action value for each entry $Q(s,a),\forall (s,a)$. At each time step $t$, we are in state $s_t$, then we choose action based on $\epsilon-$greedy strategy. With prob $\epsilon$, choose action uniformly random; with prob $1-\epsilon$, choose action based on $$a_t = \arg\max_a Q(s_t,a)$$ 

We then get the instant reward $r_t$, update the Q-table using the following rule

$$Q(s_t,a_t) \leftarrow (1-\alpha)Q(s_t,a_t) + \alpha (r_t + \max_a \gamma Q(s_{t+1},a))$$

where $\alpha \in (0,1)$ is learning rate. The algorithm is shown to converge in tabular cases. However, in cases where we cannot keep a table for state and action, we need function approximation. Consider using neural network with parameter $\theta$, the network takes as input state $s$ and action $a$. (*there are alternative parameterizations here*). Let $Q_\theta(s,a)$ be the output of the network, to estimate the optimal action value function in state $s$ and take action $a$ (and follow optimal policy thereafter). 

$$Q_\theta(s,a) \approx Q^\ast(s,a)$$

### Bellman optimality equation

We will use Bellman optimality equation to find $\theta$ such that the above approximation holds better. Recall that for optimal Q function $Q^\ast(s,a)$ the following holds for all $(s,a)$

$$Q^\ast(s_t,a_t) = \max_a \mathbb{E}\big[r_t + \gamma Q^\ast(s_{t+1},a)\big]$$

Hence a natural objective to consider is 

$$\min_\theta\  (Q_\theta(s_t,a_t) - \max_a \mathbb{E}\big[r_t + \gamma Q_\theta(s_{t+1},a)\big])^2$$

Let us first build a neural network $Q_\theta(s,a)$ as required above.

In [3]:
# define neural net Q_\theta(s,a) as a class

class Qfunction(keras.Model):
    
    def __init__(self, obssize, actsize, hidden_dims):
        """
        obssize: dimension of state space
        actsize: dimension of action space
        hidden_dims: list containing output dimension of hidden layers 
        """
        super(Qfunction, self).__init__()

        # Layer weight initializer
        initializer = keras.initializers.RandomUniform(minval=-1., maxval=1.)

        # Input Layer
        self.input_layer = keras.layers.InputLayer(input_shape=(obssize,))
        
        # Hidden Layer
        self.hidden_layers = []
        for hidden_dim in hidden_dims:
            # TODO: define each hidden layers
            layer = keras.layers.Dense(hidden_dim, activation='relu',
                                      kernel_initializer=initializer)
            self.hidden_layers.append(layer) 
        
        # Output Layer : 
        # TODO: Define the output layer.
        self.output_layer = keras.layers.Dense(actsize) 
    
    @tf.function
    def call(self, states):
        ########################################################################
        # TODO: You SHOULD implement the model's forward pass
        o = self.input_layer(states)
        for layer in self.hidden_layers:
            o = layer(o)
        qvalues = self.output_layer(o)
        return qvalues
        ########################################################################

In [4]:
# Wrapper class for training Qfunction and updating weights (target network) 

class DQN(object):
    
    def __init__(self, obssize, actsize, hidden_dims, optimizer):
        """
        obssize: dimension of state space
        actsize: dimension of action space
        optimizer: 
        """
        self.qfunction = Qfunction(obssize, actsize, hidden_dims)
        self.optimizer = optimizer
        self.obssize = obssize
        self.actsize = actsize

    def _predict_q(self, states, actions):
        """
        states represent s_t
        actions represent a_t
        """
        ########################################################################
        # TODO: Define the logic for calculate  Q_\theta(s,a)
        q_pred = self.qfunction(states)
        q_pred = tf.gather(q_pred, actions, batch_dims=1)
        return q_pred
        ########################################################################
        

    def _loss(self, Qpreds, targets):
        """
        Qpreds represent Q_\theta(s,a)
        targets represent the terms E[r+gamma Q] in Bellman equations

        This function is OBJECTIVE function
        """
        return tf.math.reduce_mean(tf.square(Qpreds - targets))


    def compute_Qvalues(self, states):
        """
        states: numpy array as input to the neural net, states should have
        size [numsamples, obssize], where numsamples is the number of samples
        output: Q values for these states. The output should have size 
        [numsamples, actsize] as numpy array
        """
        inputs = np.atleast_2d(states.astype('float32'))
        return self.qfunction(inputs)


    def train(self, states, actions, targets):
        """
        states: numpy array as input to compute loss (s)
        actions: numpy array as input to compute loss (a)
        targets: numpy array as input to compute loss (Q targets)
        """
        with tf.GradientTape() as tape:
            Qpreds = self._predict_q(states, actions)
            loss = self._loss(Qpreds, targets)
        variables = self.qfunction.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return loss

    def update_weights(self, from_network):
        """
        We need a subroutine to update target network 
        i.e. to copy from principal network to target network. 
        This function is for copying  𝜃←𝜃target 
        """
        
        from_var = from_network.qfunction.trainable_variables
        to_var = self.qfunction.trainable_variables
        
        for v1, v2 in zip(from_var, to_var):
            v2.assign(v1)

Now that we have $Q_\theta(s,a)$ we can execute policies in the environment as follows ($\epsilon-$greedy).

In [5]:
# # just pseudocode
# # Please comment out this cell when implementing the code
# raise ValueError("cannot attemp to run pseudocode")

env = gym.make('CartPole-v0')
epsilon = .1

obs, _ = env.reset()
done = False
rewardsum = 0
dqn = DQN(4, 2, [10, 5], optimizer=keras.optimizers.Adam(learning_rate=0.001))

while not done:
    if np.random.rand() < epsilon:
        action = env.action_space.sample()
    else:
        Q = dqn.compute_Qvalues(obs)
        action = np.argmax(Q)  # need some tweeking of code here
    
    obs, reward, done, info, _ = env.step(action)
    rewardsum += reward
    
print("reward under current policy {:.4f} ".format(rewardsum)) 

c:\Users\simon\.virtualenvs\rl_2023spring-9SpTBrNN\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
c:\Users\simon\.virtualenvs\rl_2023spring-9SpTBrNN\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


reward under current policy 10.0000 


c:\Users\simon\.virtualenvs\rl_2023spring-9SpTBrNN\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


We can hence collect a bunch of samples $(s_t,a_t,r_t,s_{t+1})$, and compute the targets using the current network. Let the target be $d_i$ as the $i$th target

$$d_i = \max_a r_t + \gamma Q_\theta(s_{t+1},a)$$

And then feed this value into the computational graph and minimize the Bellman error loss. This procedure has been shown to be fairly unstable. The reference paper has offered two techniques to stabilize the training process: target network and replay buffer.

**Replay Buffer**
Maintain a buffer $R$ to store trainsition tuples $(s_t,a_t,r_t,s_{t+1})$, when we minimize the Bellman error. When optimizing the Bellman error loss, we sample batches from the replay buffer and compute gradients for update on these batches. In particular, in each update, we sample $N$ tuples from buffer $(s_t,a_t,r_t,s_{t+1}) \sim R$ and then compute
loss 

$$\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) - \max_a (r_i + \gamma Q_\theta(s_i^\prime,a))^2$$

and update parameters using backprop.

**Target Network**
Maintain a target network in addition to the original pricipal network. The target network is just a copy of the original network but the parameters are not updated by gradients. The target network $\theta_{\text{target}}$ is copied from the principal network every $\tau$ time steps. Target network is used to compute the targets for update

$$d_i = \max_a r_t + \gamma Q_{\theta^{-}}(s_{i}^\prime,a)$$

the targets are used in the loss function to update the principal network parameters. This slowly updated target network ensures that the targets come from a relatively stationary distribution and hence stabilize learning.

Hence several critical parts of the complete pseudocode for DQN is as follows:

**Initialization**: principal network $Q_\theta(s,a)$, target network $Q_{\theta^{-}}(s,a)$. Replay buffer $R = \{\}$ (empty). 

**At each time step $t$**: execute action using $\epsilon-$greedy based on the principal network $Q_\theta(s,a)$. To update $\theta$: sample $N$ tuples $(s_i,a_i,r_i,s_i^\prime) \sim R$, compute empirical loss 

$$\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) - \max_a (r_i + \gamma Q_{\theta^{-}}(s_i^\prime,a))^2$$

and update parameter $\theta$ using backprop (just take one gradient step).

**Update target network**: Every $\tau$ time steps, update target network by copying $\theta_{\text{target}} \leftarrow \theta$.

In [6]:
# Implement replay buffer
class ReplayBuffer(object):
    
    def __init__(self, maxlength):
        """
        maxlength: max number of tuples to store in the buffer
        if there are more tuples than maxlength, pop out the oldest tuples
        """
        self.buffer = deque()
        self.number = 0
        self.maxlength = maxlength
    
    def append(self, experience):
        """
        this function implements appending new experience tuple
        experience: a tuple of the form (s,a,r,s^\prime)
        """
        self.buffer.append(experience)
        self.number += 1
        if(self.number > self.maxlength):
            self.pop()
        
    def pop(self):
        """
        pop out the oldest tuples if self.number > self.maxlength
        """
        while self.number > self.maxlength:
            self.buffer.popleft()
            self.number -= 1
    
    def sample(self, batchsize):
        """
        this function samples 'batchsize' experience tuples
        batchsize: size of the minibatch to be sampled
        return: a list of tuples of form (s,a,r,s^\prime)
        """
        inds = np.random.choice(len(self.buffer), batchsize, replace=False)
        return [self.buffer[idx] for idx in inds]
        

Now that we have all the ingredients for DQN, we can write the main procedure to train DQN on a given environment. The implementation is straightforward if you follow the pseudocode. Refer to the pseudocode pdf for details.

In [7]:
################################################################################
# TODO: Set the required parameters. All parameters can be tuned by yourself.
from tqdm.notebook import tqdm  # need to install ipywidgets
lr = 0.0015 # 5e-3  # learning rate for gradient update 
batchsize = 64  # batchsize for buffer sampling
maxlength = 10000  # max number of tuples held by buffer
envname = "CartPole-v0"  # environment name
tau = 400  # time steps for target update
episodes = 5000  # number of episodes to run
initialize = 100  # initial time steps before start updating
# epsilon = .05  # constant for exploration
epsilon = 0.5
epsilon_min = 0.01
epsilon_decay = 0.995
gamma = 0.95 # discount
hidden_dims=[32, 32] # hidden dimensions
################################################################################

# initialize environment
env = gym.make(envname)
obssize = env.observation_space.low.size
actsize = env.action_space.n

# optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr)

# initialize networks
Qprincipal = DQN(obssize, actsize, hidden_dims, optimizer)
Qtarget = DQN(obssize, actsize, hidden_dims, optimizer)

# initialization of buffer
buffer = ReplayBuffer(maxlength)

################################################################################
# TODO: Complete the main iteration
# CartPole-v0 defines "solving" as getting average reward of 195.0 over 100 consecutive trials.

rrecord = []
loss = None
totalstep = 0
global_logger = tqdm(leave=False)
for ite in range(episodes):

    obs, _ = env.reset()
    done = False
    rsum = 0
    epsilon = max(epsilon * (epsilon_decay**ite), epsilon_min)
    global_logger.set_description(f"[{ite:05}] epsilon: {epsilon:.6f}")
    while not done:
        
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            q_values_principal = Qprincipal.compute_Qvalues(obs)
            action = np.argmax(q_values_principal)

        next_obs, reward, done, *_ = env.step(action)
        buffer.append((obs, action, reward, next_obs, done))  # append s,a,r,s',done

        rsum += reward
        
        # Update
        if totalstep >= initialize:
            batch = buffer.sample(batchsize)
            s, a, r, s_prime, ds = map(np.array, list(zip(*batch)))
            
            q_values_target = Qtarget.compute_Qvalues(s_prime)
            target = r + gamma * np.amax(q_values_target, axis=1) * (1 - ds)

            loss = Qprincipal.train(s, a, target)

        if totalstep % tau == 0:
            Qtarget.update_weights(Qprincipal)

        obs = next_obs
        totalstep += 1

        log_str = f'[{totalstep}] R={np.mean(rrecord[-10:]):.4f}'
        if loss is not None:
            log_str += f', L={loss:.6f}'
            
        global_logger.set_postfix_str(log_str)

    global_logger.update(1)
################################################################################
    
    ## DO NOT CHANGE THIS PART!
    rrecord.append(rsum)
    if ite % 10 == 0:
        print('iteration {} ave reward {}'.format(ite, np.mean(rrecord[-10:])))

    ave100 = np.mean(rrecord[-100:])   
    if  ave100 > 195.0:
        print("Solved after %d episodes."%ite)
        break

0it [00:00, ?it/s]

c:\Users\simon\.virtualenvs\rl_2023spring-9SpTBrNN\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\simon\.virtualenvs\rl_2023spring-9SpTBrNN\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


iteration 0 ave reward 13.0
iteration 10 ave reward 14.2
iteration 20 ave reward 52.1
iteration 30 ave reward 49.2
iteration 40 ave reward 74.2
iteration 50 ave reward 79.1
iteration 60 ave reward 61.4
iteration 70 ave reward 105.3
iteration 80 ave reward 147.9
iteration 90 ave reward 151.0
iteration 100 ave reward 165.7
iteration 110 ave reward 184.6
iteration 120 ave reward 171.6
iteration 130 ave reward 150.3
iteration 140 ave reward 171.9
iteration 150 ave reward 199.8
iteration 160 ave reward 187.4
iteration 170 ave reward 205.0
iteration 180 ave reward 166.7
iteration 190 ave reward 160.2
iteration 200 ave reward 141.5
iteration 210 ave reward 179.1
iteration 220 ave reward 110.4
iteration 230 ave reward 40.8
iteration 240 ave reward 190.4
iteration 250 ave reward 169.1
iteration 260 ave reward 169.6
iteration 270 ave reward 186.4
iteration 280 ave reward 220.1
iteration 290 ave reward 203.0
iteration 300 ave reward 135.2
iteration 310 ave reward 152.5
iteration 320 ave reward 17

In [1]:
# plot [episode, reward] history
x = [i+1 for i in range(len(rrecord))]
plt.plot(x, rrecord)
plt.title('episode rewards')
plt.xlabel('episodes')
plt.ylabel('rewards')
plt.show()

NameError: name 'rrecord' is not defined

Once you run the code, save the notebook file of name "Lab 3_your name.ipynb" and submit on eTL.

# Do NOT submit as zip files.